# `XGBoost` 예측모형 자동화

`XGBoost` 예측모형은 뛰어난 성능으로 이미 캐글 등 대다수 경진대회를 휩쓴 검증된 알고리즘이다. 그렇다고 모든 예측문제에 기계학습 알고리즘으로 선택을 할 수는 없다. 우선, 뛰어난 성능을 가진 XGBoost 알고리즘의 마지막까지 높일 수 있는 초모수(Hyper Paramter) 튜닝에 대해서 살펴보자.

<img src="fig/xgboost-python.png" alt="XGBoost" width="77%" />

## 초모수(Hypter parameter) 튜닝

XGBoost와 관련된 Hypter Parameter는 다음 세가지 범주로 나뉜다.  XGBoost가 의사결정나무에 기반하기 때문에 관련된 모수와 기계학습과 관련이 있기 때문에 이와 연관된 다수 초모수가 포함된다.

- 학습 초모수
    - `objective` [기본설정값=reg:linear]: 지도학습 손실 최소화 함수를 정의
        - `binary:logistic`: 이항 분류 문제 로직스틱 회귀모형으로 반환값이 클래스가 아니라 예측 확률.
        - `multi:softmax`: 다항 분류 문제의 경우 소프트맥스(Softmax)를 사용해서 분류하는데 반횐되는 값이 예측확률이 아니라 클래스임. 또한 `num_class`도 지정해야함.
        - `multi:softprob`: 각 클래스 범주에 속하는 예측확률을 반환함.
    - `eval_metric`: 설정한 `objective`별로 기본설정값이 지정되어 있음.
        - rmse: root mean square error
        - mae: mean absolute error
        - logloss: negative log-likelihood
        - error: Binary classification error rate (0.5 threshold)
        - merror: Multiclass classification error rate
        - mlogloss: Multiclass logloss
        - auc: Area under the curve
    - `seed` [기본설정값: 0]: 재현가능하도록 난수를 고정시킴.
- 일반 초모수
    - booster: 의사결정 기반 모형(`gbtree`), 선형 모형(`linear`)
    - mthread: 병렬처리에 사용되는 코어수, 특정값을 지정하지 않는 경우 자동으로 시스템 코어수를 탐지하여 병렬처리에 동원함.
- 부스팅 초모수
    - `eta` [기본설정값: 0.3]: GBM에 학습율과 유사하고 일반적으로 0.01 ~ 0.2 값이 사용됨
    - `min_child_weight` [기본설정값: 1]: 과적합(overfitting)을 방지할 목적으로 사용되는데, 너무 높은 값은 과소적합(underfitting)을 야기하기 때문에 CV를 사용해서 적절한 값이 제시되어야 한다.
    - `max_depth` [기본설정값: 6]: 과적합 방지를 위해서 사용되는데 역시 CV를 사용해서 적절한 값이 제시되어야 하고 보통 3-10 사이 값이 적용된다.
    - `max_leaf_nodes`: `max_leaf_nodes` 값이 설정되면 `max_depth`는 무시된다. 따라서 두값 중 하나를 사용한다.
    - `max_delta_step` [기본설정값: 0]: 일반적으로 잘 사용되지 않음.
    - `subsample` [기본설정값: 1]: 개별 의사결정나무 모형에 사용되는 임의 표본수를 지정. 보통 0.5 ~ 1 사용됨.
    - `colsample_bytree` [기본설정값: 1]: 개별 의사결정나무 모형에 사용될 변수갯수를 지정. 보통 0.5 ~ 1 사용됨.
    - `colsample_bylevel` [기본설정값: 1]: `subsample`, `colsample_bytree` 두 초모수 설정을 통해서 이미 의사결정나무 모형 개발에 사용될 변수갯수와 관측점 갯수를 사용했는데 추가로 `colsample_bylevel`을 지정하는 것이 특별한 의미를 갖는지 의문이 듦.
    - `lambda` [기본설정값: 1]: 능선 회쉬(Ridge Regression)의 L2 정규화(regularization) 초모수. 그다지 많이 사용되고 있지는 않음.
    - `alpha` [기본설정값: 0]: 라쏘 회귀(Lasso Regression)의 L1 정규화(regularization) 초모수로 차원이 높은 경우 알고리즘 속도를 높일 수 있음.
    - `scale_pos_weight` [기본설정값: 1]: 클래스 불균형이 심한 경우 0보다 큰 값을 지정하여 효과를 볼 수 있음.


참고문헌: [Analytics Vidhya, "Complete Guide to Parameter Tuning in XGBoost with codes in Python"](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)


# XGBoost 초모수 확인

XGBoost 초모수(Hyper Parameter)는 XGBoost 객체를 하나 생성한 후에 `print()` 함수로 확인이 가능하다. 그렇다고 모든 Hyper parameter가 중요한 것은 아니다. 

In [2]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier()

print(xgb_clf)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


## XGBoost 초모수 헬로월드

XGBoost 초모수를 간단한 헬로월드 코드를 작성해본다.

In [7]:
# 경고 출력하지 않음 -----------
import warnings
warnings.filterwarnings('ignore')

# 라이브러리와 데이터 가져오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn
from sklearn.metrics import classification_report, accuracy_score

print('sklearn version: %s' % sklearn.__version__)
print('xgboost version: %s' % xgb.__version__)

hr_df = pd.read_csv("data/HR_comma_sep.csv")
hr_df.head()

# 시험/훈련 데이터 관측점 구분
colnames = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 
       'promotion_last_5years']

X, y = hr_df[colnames], hr_df[['left']]
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=777)

# XGBoost 예측모형
xgb_model = xgb.XGBClassifier(silent=False, 
                              booster='gbtree',
                              scale_pos_weight=1,
                              learning_rate=0.01,  
                              colsample_bytree = 0.4,
                              subsample = 0.8,
                              objective='binary:logistic', 
                              n_estimators=100, 
                              max_depth=4, 
                              gamma=10, 
                              seed=777)

hr_pred = xgb_model.fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, hr_pred))

sklearn version: 0.21.2
xgboost version: 0.90
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      3440
           1       0.99      0.68      0.80      1060

    accuracy                           0.92      4500
   macro avg       0.95      0.84      0.88      4500
weighted avg       0.93      0.92      0.92      4500



## 초모수 1개 대상 예측모형 구축

`max_depth` 의사결정나무 모형 깊이 초모수를 달리해서 XGBoost 예측모형의 성능을 비교해본다.

In [11]:
accuracy_list = []

max_depth_list = [3,5,7,9,10]

for max_depth in max_depth_list:
    xgb_model = xgb.XGBClassifier(max_depth=max_depth, seed=777)
    xgb_pred = xgb_model.fit(X_train, y_train).predict(X_test)
    xgb_accuracy = accuracy_score(y_test, xgb_pred) 
    accuracy_list.append(xgb_accuracy)
    
xgb_df = pd.DataFrame({'tree depth':max_depth_list, 'accuracy':accuracy_list})
xgb_df.head()    

,tree depth,accuracy
0,3,0.974889
1,5,0.979333
2,7,0.982222
3,9,0.983556
4,10,0.984889
